In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
from datetime import date

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

import numpy as np
import sys
from scipy.optimize import root_scalar
from functools import partial

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2
    
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
# Importamos el archivo Excel a un DataFrame de pandas
df = pd.read_excel('../data/SOFR-10012019-10292020.xls',"nice format")

df = df.drop(df.columns[[1]], axis='columns')


In [5]:

# Los campos de fecha se importan como datetime (con horas, minutos y segundos).
# Se mantiene sólo la fecha.

df['fecha_inicial'] = df['DATE'] #En la Clase 2 IN[10] se agrega una nueva columna de fechas
df['TASA_SOFR'] = df['RATE\n(PERCENT)'] /100.0 # Transformamos las tasas de intereses creando una nueva columna tasa/100

In [6]:

#construcción fecha final 
fecha = df.iloc[0:269,0] #Llamo todas las filas de matriz df en la primera columna 
addfecha = "2020-10-29" #se agrega la fecha
fecha = np.append(addfecha, fecha) #se fusionan en una sola matrix

df["fecha_final"]= pd.Series(fecha) #extrayendo la fecha a la matriz df

df['fecha_inicial'] = pd.to_datetime(df['fecha_inicial'], errors='coerce') 
df['fecha_final'] = pd.to_datetime(df['fecha_final'], errors='coerce')

df['fecha_inicial'] = df['fecha_inicial'].dt.date
df['fecha_final'] = df['fecha_final'].dt.date

#eliminando columnas restantes del data frame
df = df.drop(df.columns[[1]], axis='columns')
df = df.drop(df.columns[[0]], axis='columns') 

#se da vuelta la matriz
df= df.iloc[::-1] 

# tabulación del resultado
df.style.format({'RATE(PERCENT)': '{0:.2%}'}) #grafica el df en forma bonita

,fecha_inicial,TASA_SOFR,fecha_final
269,2019-10-01,0.018800,2019-10-02
268,2019-10-02,0.018500,2019-10-03
267,2019-10-03,0.018400,2019-10-04
266,2019-10-04,0.018200,2019-10-07
265,2019-10-07,0.018300,2019-10-08
264,2019-10-08,0.018500,2019-10-09
263,2019-10-09,0.018500,2019-10-10
262,2019-10-10,0.018500,2019-10-11
261,2019-10-11,0.018500,2019-10-15
260,2019-10-15,0.020000,2019-10-16


In [7]:

fixing = Qcf.time_series()
accrual_date: Qcf.QCDate
final = Qcf.time_series()  



In [8]:
class OisCashFlow:
    
    # AD: Esto no era necesario.
    
    def __init__ (self, start_date, end_date, settlement_date, notional, currency, amortization,amort_is_cashflow,interest_rate, on_index, spread, gearing):
        
        self.start_date = start_date
        self.end_date = end_date
        self.settlement_date = settlement_date
        self.notional = notional
        self.currency = currency
        self.amortization = amortization
        self.amort_is_cashflow = amort_is_cashflow
        self.interest_rate = interest_rate
        self.on_index = on_index
        self.spread = spread
        self.gearing = gearing

    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixing: Qcf.time_series) -> float:
        
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
                 
        self.fecha_final = accrual_date
        self.diff = self.start_date.day_diff(self.fecha_final)

        for i in range(270):
            
                        count = i

                        fec_ini_1 = Qcf.QCDate(df.iloc[i]['fecha_inicial'].day, df.iloc[i]['fecha_inicial'].month, df.iloc[i]['fecha_inicial'].year)
                
                        fec_fin_ = Qcf.QCDate(df.iloc[i]['fecha_final'].day, df.iloc[i]['fecha_final'].month, df.iloc[i]['fecha_final'].year)
                        

                        if fec_ini_1 == self.start_date:

                             for j in range(count,count+self.diff):

                                fec_ini_2 = Qcf.QCDate(df.iloc[j]['fecha_inicial'].day, df.iloc[j]['fecha_inicial'].month, df.iloc[j]['fecha_inicial'].year)

                                fec_fin_2 = Qcf.QCDate(df.iloc[j]['fecha_final'].day, df.iloc[j]['fecha_final'].month, df.iloc[j]['fecha_final'].year)


                                fixing[fec_ini_2] = df.iloc[j]['TASA_SOFR']
                                final[fec_ini_2] = df.iloc[j]['TASA_SOFR']

        asoc_inicial = Qcf.time_series_dates(fixing)                         
        asoc_final = Qcf.time_series_dates(final)                                                         
        asoc_tasa = Qcf.time_series_values(fixing)

        df2= pd.DataFrame({'FECHA INICIAL':asoc_inicial,'FECHA FINAL':asoc_final,'TASAS':asoc_tasa})
        
        #generando función auxiliar

        def get_wf(fecha_inicial: Qcf.QCDate, fecha_final: Qcf.QCDate, valor_tasa: float) -> float:
            
            self.qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
            
            return self.qc_tasa.wf(fecha_inicial, fecha_final)

        df2['wf'] = df2.apply(
        lambda row: get_wf(row['FECHA INICIAL'], row['FECHA FINAL'],row['TASAS']),
        axis=1
        )

        df2.style.format({'TASAS': '{0:.2%}', 'wf': '{0:.6%}'})
        
        #factores de capitalización compuesto

        self.prod = df2['wf'].product()
        self.dias = self.start_date.day_diff(self.fecha_final)
                        # Tasa equivalente
                        # prod = 1 + tasa_eq * dias / 360
        self.tasa_eq = (self.prod - 1) * 360.0 / self.dias

        return self.tasa_eq

        #def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        #self.interes_flot_ois = self.notional * self.tasa_eq * self.dias / 360     
        #return self.interes_flot_ois  
        #TRATAMOS DE EFECTUAR ESTA FUNCIÓN PERO NO PUDIMOS 

In [9]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

In [10]:
ois.get_accrued_rate(Qcf.QCDate(1,5,2020),fixing)

#la tasa equivalente a la fecha 1-05-2020 corresponde a un 1,09% aprox. 

NameError: name 'ois' is not defined

## Tests

In [11]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [12]:
amount_tol = 10000
rate_tol = .001

In [13]:
exitos = 0

In [14]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [16]:
ois = OisCashFlow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [17]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [18]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [19]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [20]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [21]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

ValueError: arrays must all be same length

Se verificará el cálculo usando `df_fixings`.

In [22]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [23]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [24]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [25]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [26]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [27]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [28]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [29]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [30]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [31]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [32]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [33]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

NameError: name 'accrued_rate' is not defined

In [34]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

NameError: name 'accrued_rate' is not defined

### `OisCashflow.accrued_interest`

In [35]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

AttributeError: 'OisCashFlow' object has no attribute 'get_accrued_interest'

In [36]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

NameError: name 'accrued_rate' is not defined

In [37]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

NameError: name 'accrued_interest' is not defined

### `OisCashflow.amount`

In [38]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

AttributeError: 'OisCashFlow' object has no attribute 'amount'

In [39]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [40]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

NameError: name 'amount' is not defined

In [41]:
suma_exito(amount, check_amount, 'monto')

NameError: name 'amount' is not defined

### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [42]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [43]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [44]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [45]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [46]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'set_expected_rate' is not defined

In [47]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'set_expected_rate' is not defined

In [48]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'set_expected_rate' is not defined

In [49]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'set_expected_rate' is not defined

In [50]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

NameError: name 'set_expected_rate' is not defined

### `present_value`

In [51]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'set_expected_rate' is not defined

In [52]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'set_expected_rate' is not defined

In [53]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'set_expected_rate' is not defined

In [54]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'set_expected_rate' is not defined

In [55]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

NameError: name 'set_expected_rate' is not defined

In [56]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 0
